# 4b. Create K-means Clustered Input Data

I used the method from the K means exploration to find the cluster that intersect with the prediction and union with the radarsat ground truth. I didn't use any multiprocessing this time because I will already be using all cores for K-means clustering instead of spliting up the work of making masks.

## Setup 

In [1]:
import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import shapely.geometry

import re
import numpy as np 

import matplotlib.pyplot as plt
import os
import re
import cv2
from scipy.misc import imresize
from u_net_functions import dice_coef, dice_coef_loss, jacc_coef, jacc_coef_loss, jacc_coef_int, get_unet
from resnet_functions import fcn_model
from sklearn.cluster import KMeans

import tensorflow as tf
print(tf.__version__)

%matplotlib inline

Using TensorFlow backend.


1.2.0


In [2]:
model = get_unet()
model.load_weights('unet_checkpoints_1/weights.78-0.13961.hdf5')
#This is an initialization for K-means instead of using K++. I got this from a diverse image fro mthe training sample.
init_center = np.array([[ 58,  79,  41],
                       [ 77,  77,  54],
                       [ 73,  96,  53],
                       [ 50,  68,  36],
                       [ 65,  87,  47],
                       [ 40,  56,  29],
                       [ 71,  73,  50],
                       [ 87, 101,  64]], dtype='uint8')

## Helper Functions

In [3]:
def invert(img):
    return (1-img)
def resize(img, new_shape):
    img_resized = np.zeros(new_shape+(img.shape[2],)).astype('float32')
    for i in range(img.shape[2]):
        img_resized[:, :, i] = imresize(img[:, :, i], new_shape, interp='bicubic')
    return img_resized

def scale_bands(img, lower_pct = 1, upper_pct = 99):
    """
    Rescale the bands of a multichannel image for display
    """
    # Loop through the image bands, rescaling each one
    img_scaled = np.zeros(img.shape, np.uint8)
    
    for i in range(img.shape[2]):
        
        band = img[:, :, i]
        
        # Pick out the lower and upper percentiles
        lower, upper = np.percentile(band, [lower_pct, upper_pct])
        
        # Normalize the band
        band = (band - lower) / (upper - lower) * 255
        
        # Clip the high and low values, and cast to uint8
        img_scaled[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
        
    return img_scaled

from sklearn.metrics import jaccard_similarity_score
# Function to create an iterator with examples
smooth=1
def jacc_coef_int(y_true, y_pred):
    y_pred_pos = y_pred
    intersection = np.sum(y_true * y_pred_pos)
    sum_ = np.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return np.mean(jac)

def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

## Main Function  

In [4]:
def make_files(files, input_size,init_center):
    num = 0
    for tile_no in files:
        #Loading and resizing Image
        img = np.load(tile_no + '_img.npy') 
        img = resize(img, (input_size, input_size))
        X = img.astype('float32')
        X = (X / X.max() - 0.5) * 2
        Z = img.reshape((-1,3))
        Z = np.float32(Z)

        #Loading and resizing Radar Ground Truth Masks
        mask = np.load(tile_no + '_mask.npy')
        Y = imresize(mask, (input_size, input_size))
        Y = Y.astype('float32') / 255

        #Runing predictions with U-Net
        predict_unet = model.predict(X[None, ...])[0, ...] > 0.15
        predict_unet = predict_unet * 1
        predict_unet = predict_unet[:, :, 0]

        K_intersect_P = dict()
        K_intersect_R = dict()
        K_segment = dict()
        K_overlap = dict()

        K = 8
        for k in range(4,K):
            kmeans = KMeans(n_clusters=k, n_jobs=-1,init=init_center[:k]).fit(Z)
            label = kmeans.labels_
            center = kmeans.cluster_centers_
            center = np.uint8(center)

            segment_list = []
            total_intersect_list_P = []
            overlap_list = []

            total_intersect_list_R = []

            for cluster in range(0,k):
                label_mask = label.flatten()==cluster
                res = center[:,0][label_mask*1]
                res2 = res.reshape((input_size,input_size,))
                segment = cv2.normalize(res2,dst=np.zeros(shape=(input_size,input_size)),alpha=0,beta=1,norm_type=cv2.NORM_MINMAX)
                #if cluster == 0:
                #    segment = invert(segment)
                segment_list.append(segment)

                intersection = np.sum(predict_unet.flatten() * segment.flatten())
                total_intersect_list_P.append(intersection)
                overlap_list.append((predict_unet.flatten()*segment.flatten()).reshape((input_size,input_size,)))

                intersection = np.sum(Y.flatten() * segment.flatten())
                total_intersect_list_R.append(intersection)

            index_P = np.argmax(total_intersect_list_P)   
            index_R = np.argmax(total_intersect_list_R)
            cluster_mask = segment_list[index_P] #Just cluster mask
            overlap_mask = overlap_list[index_P] #Intersection of cluster and prediction

            if index_P == index_R:
                K_intersect_P[k] = max(total_intersect_list_P)
                K_intersect_R[k] = max(total_intersect_list_R)
                K_segment[k] = cluster_mask
                K_overlap[k] = overlap_mask 

        if len(K_intersect_P) > 0:
            best_K = max(K_intersect_R, key=K_intersect_R.get)
            #Intersect wiht prediction and union with RADARSAT
            new_cluster_mask = np.array([1 if pixel==2 else 1 if pixel == 1 else 0 for pixel in ((K_overlap[best_K]).flatten()+Y.flatten())]).reshape((input_size,input_size)).astype('uint8')
            #new_cluster_mask = K_overlap[best_K]

            #Make file
            np.save("training_tiles_clustered/%d_mask"%num, new_cluster_mask)
            np.save("training_tiles_clustered/%d_img"%num, img)
            num += 1


## Generating Masks 

In [ ]:
input_size = 256
#os.makedirs('training_tiles2')

dir_path = 'training_tiles_1000size/'
np_files = [os.path.join(path,f[:f.find('_img.npy')])
             for path,_,files in os.walk(dir_path) 
             for f in files if (f.endswith('img.npy'))]

if not os.path.exists('training_tiles_clustered'):
    os.makedirs('training_tiles_clustered')


make_files(np_files, input_size,init_center)

/home/mlt/anaconda2/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
